In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
!pip install bitsandbytes transformers datasets huggingface_hub auto-gptq peft accelerate optimum
!huggingface-cli login


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 984.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.4.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
# Load model
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mistralai/Mistral-7B-Instruct-v0.3", #"unsloth/Meta-Llama-3-8B",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)
model.print_trainable_parameters()

==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.43.4.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


trainable params: 41,943,040 || all params: 7,289,966,592 || trainable%: 0.5754


In [ ]:
# def get_scenario_text_dilemmas(row):
#     """
#     Extracts scenario text from the dilemmas dataset row and calculates the probabilities
#     of each action being less ethical based on the provided scores.

#     Args:
#     - row (dict): A row from the dilemmas dataset containing actions and labels.

#     Returns:
#     - tuple: A formatted scenario text prompt and the assistant message with calculated probabilities.
#     """

#     actions = row.get('actions', [])
#     if len(actions) < 2:
#         return "", ""

#     action1_description = actions[0].get('description', '')
#     action2_description = actions[1].get('description', '')


#     scenario_text = f"Action 1: {action1_description}\nAction 2: {action2_description}\n\n"
#     answer_prompt = "Given the actions above, determine which action is less ethical. Provide probabilities for each action being less ethical:\n\n"


#     gold_annotations = row.get('gold_annotations', [0, 0])
#     yes_value = gold_annotations[0]
#     no_value = gold_annotations[1]


#     total = yes_value + no_value
#     if total > 0:
#         yes_probability = yes_value / total
#         no_probability = no_value / total
#     else:
#         yes_probability = 0.0
#         no_probability = 0.0


#     assistant_message = f"Yes: {yes_probability:.2f}, No: {no_probability:.2f}"


#     return scenario_text + answer_prompt, assistant_message


In [ ]:
# def apply_template_dilemmas(batch):
#     """
#     Applies the chat template to each row in the batch of dilemmas data.

#     Args:
#     - batch (dict): A batch of rows from the dilemmas dataset.

#     Returns:
#     - dict: A dictionary with formatted messages ready for tokenization.
#     """
#     batch_texts = []
#     for i in range(len(batch['actions'])):
#         row = {key: batch[key][i] for key in batch.keys()}

#         scenario_text, assistant_message = get_scenario_text_dilemmas(row)

#         message = [{"role": "user", "content": scenario_text},
#                    {"role": "assistant", "content": assistant_message}]

#         try:
#             formatted_message = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=False)
#             batch_texts.append(formatted_message)
#         except Exception as e:
#             print(f"Error processing message: {message}")
#             raise e

#     return {"text": batch_texts}


In [7]:
def process_and_format_data(batch):
    batch_texts = []
    for i in range(len(batch['id'])):
        row = {key: batch[key][i] for key in batch.keys()}

        # Extract scenario text
        action1 = row['actions'][0]['description']
        action2 = row['actions'][1]['description']
        scenario_text = f"Action 1: {action1}\n\nAction 2: {action2}\n\n"

        answer_prompt = (
        "You are an agent designed to compare two possible actions taken by individuals in moral scenarios encountered in day-to-day life. "
    "Given the scenario, evaluate whether the first action is less ethical or reasonable based on the details provided. "
    "Provide a probability for the label 'Yes,' indicating that the first action is less aligned with ethical or reasonable standards compared to the second action."
    )

        gold_annotations = row['gold_annotations']
        less_ethical_counts = gold_annotations
        total_responses = sum(less_ethical_counts)
        if total_responses > 0:
          yes_probability = less_ethical_counts[0] / total_responses  # Probability of action 1 being less ethical

        assistant_message = f"Yes: {yes_probability:.2f}"

        # Create the message with proper roles
        message = [{"role": "user", "content": scenario_text +"\n"+ answer_prompt},
                   {"role": "assistant", "content": assistant_message}]

        try:
            formatted_message = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

            # Ensure the formatted message is a string
            if isinstance(formatted_message, list):
                formatted_message = ' '.join(formatted_message)
            elif not isinstance(formatted_message, str):
                formatted_message = str(formatted_message)

            batch_texts.append(formatted_message)
        except Exception as e:
            print(f"Error processing message: {message}")
            raise e

    return {"text": batch_texts}

In [8]:
import pandas as pd
from io import StringIO
# Read the JSONL file directly from its path

file_path = "/content/train.scruples-dilemmas.jsonl"
df = pd.read_json(file_path, lines=True)


# Read the file into a DataFrame
df = pd.read_json(file_path, lines=True)


In [9]:
from datasets import Dataset
import pandas as pd
df = df.sample(frac=0.8, random_state=42)
dataset = Dataset.from_pandas(df)

# Apply the template function to the dataset
processed_dataset = dataset.map(process_and_format_data, batched=True)
print(processed_dataset[0])

Map:   0%|          | 0/18877 [00:00<?, ? examples/s]

{'id': '52WFgRXGyjnrO23RJcsNNQgWukPaNdoi', 'actions': [{'description': "leaving my current d&d group for another one if they can't agree to be more consistent with scheduling", 'id': 'E1P3UQlXNr3QQKsrzUBWexwveZ2TgiBS'}, {'description': 'being the "Teachers Pet"', 'id': '42syROiY5BpP31AymraZn8TGmgPZzHdJ'}], 'gold_annotations': [1, 4], 'gold_label': 1, 'human_perf_annotations': [0, 0], 'human_perf_label': 0, 'controversial': True, '__index_level_0__': 7745, 'text': '<s>[INST] Action 1: leaving my current d&d group for another one if they can\'t agree to be more consistent with scheduling\n\nAction 2: being the "Teachers Pet"\n\n\nYou are an agent designed to compare two possible actions taken by individuals in moral scenarios encountered in day-to-day life. Given the scenario, evaluate whether the first action is less ethical or reasonable based on the details provided. Provide a probability for the label \'Yes,\' indicating that the first action is less aligned with ethical or reasonabl

In [10]:
for i in range(5):
    print(processed_dataset[i])

{'id': '52WFgRXGyjnrO23RJcsNNQgWukPaNdoi', 'actions': [{'description': "leaving my current d&d group for another one if they can't agree to be more consistent with scheduling", 'id': 'E1P3UQlXNr3QQKsrzUBWexwveZ2TgiBS'}, {'description': 'being the "Teachers Pet"', 'id': '42syROiY5BpP31AymraZn8TGmgPZzHdJ'}], 'gold_annotations': [1, 4], 'gold_label': 1, 'human_perf_annotations': [0, 0], 'human_perf_label': 0, 'controversial': True, '__index_level_0__': 7745, 'text': '<s>[INST] Action 1: leaving my current d&d group for another one if they can\'t agree to be more consistent with scheduling\n\nAction 2: being the "Teachers Pet"\n\n\nYou are an agent designed to compare two possible actions taken by individuals in moral scenarios encountered in day-to-day life. Given the scenario, evaluate whether the first action is less ethical or reasonable based on the details provided. Provide a probability for the label \'Yes,\' indicating that the first action is less aligned with ethical or reasonabl

In [11]:
import pandas as pd
from datasets import Dataset

In [12]:
import pandas as pd
from datasets import Dataset

# Load and process the dataset
file_path = "/content/train.scruples-dilemmas.jsonl"
df = pd.read_json(file_path, lines=True)
print(df.head())


subset_df = df.sample(frac=0.8, random_state=42)
dataset = Dataset.from_pandas(subset_df)
print(dataset)


processed_dataset = dataset.map(process_and_format_data, batched=True)
print(processed_dataset)

                                 id  \
0  eaN4nAeeQ2cF9kF3Ilo6XiwxKqWwYIcv   
1  KT1XEQ8R1Mfa1QvgK7kqc6cujJbnvLW5   
2  ELBHKdmOoI1e4dKzGYA5P3jsSDlPwXee   
3  bkJa8QFMwOHGbjIUuKo8W9Ie3BW5joAi   
4  OEOVomRMNijB8vlX9a9OjZ3ZLkVYOMGG   

                                             actions gold_annotations  \
0  [{'id': 'EFGAk9pzLMuCam1aaJSNfKoLHmxLIvbN', 'd...           [0, 5]   
1  [{'id': 'CggLgzSNbn1IA4JK49Q658myA3zwGQoF', 'd...           [3, 2]   
2  [{'id': 'CXdqhinKPYPZbxtItJljiTKB8eMvndCt', 'd...           [5, 0]   
3  [{'id': 'hqewDuC2un87b1SxJHT6qb3R0HtNzROZ', 'd...           [3, 2]   
4  [{'id': 'cMSwGBtuHtwFzk9Wb1VMZgncKWnKKCyK', 'd...           [4, 1]   

   gold_label human_perf_annotations  human_perf_label  controversial  
0           1                 [0, 0]                 0          False  
1           0                 [0, 0]                 0           True  
2           0                 [0, 0]                 0          False  
3           0                 [0, 0]  

Map:   0%|          | 0/18877 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'actions', 'gold_annotations', 'gold_label', 'human_perf_annotations', 'human_perf_label', 'controversial', '__index_level_0__', 'text'],
    num_rows: 18877
})


In [13]:
print(dataset.column_names)

['id', 'actions', 'gold_annotations', 'gold_label', 'human_perf_annotations', 'human_perf_label', 'controversial', '__index_level_0__']


In [14]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
# Specify the correct field in your dataset that contains the formatted string data
dataset_text_field = 'text'  # Ensure 'text' is now formatted as a single string

# data_collator = DataCollatorForCompletionOnlyLM(
#     tokenizer=tokenizer,
#     mlm=False,
#     response_template="[Completion]",
#     pad_to_multiple_of=8
# )

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    lr_scheduler_type="linear",
    warmup_steps=20,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    optim="adamw_8bit",
    weight_decay=0.01,
    seed=42,
)

# Set up the trainer with the corrected dataset formatting
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=10,  # Adjust based on your model's capabilities
)

# Start training



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/18877 [00:00<?, ? examples/s]

In [15]:
trainer.save_model("Fine_tuned_Llama")

In [16]:
def get_model_probabilities(text, model, tokenizer, device='cuda'):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]
    probs = torch.softmax(last_token_logits, dim=-1)
    return last_token_logits, probs

In [17]:
def get_yes_no_probabilities(probs, tokenizer):
    yes_token_id = tokenizer.convert_tokens_to_ids('yes')
    no_token_id = tokenizer.convert_tokens_to_ids('no')
    Yes_token_id = tokenizer.convert_tokens_to_ids('Yes')
    No_token_id = tokenizer.convert_tokens_to_ids('No')

    yes_prob = probs[0, yes_token_id].item() + probs[0, Yes_token_id].item()
    no_prob = probs[0, no_token_id].item() + probs[0, No_token_id].item()

    # Normalize the probabilities
    total_prob = yes_prob + no_prob
    if total_prob > 0:
        yes_prob = yes_prob / total_prob
        no_prob = no_prob / total_prob

    return yes_prob, no_prob

In [18]:
import torch
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [19]:
import torch
import numpy as np

def dirichlet_multinomial_loss(pred_probs, counts, alpha=1.0, eps=1e-8):
    """
    Calculate the corrected Dirichlet Multinomial Loss.

    This function computes the negative log likelihood of the Dirichlet-Multinomial
    distribution, ensuring consistency with provided probabilities and counts.

    Args:
        pred_probs (torch.Tensor or list): Predicted probabilities from the model.
            Should be a 1D tensor or list of length 2 (for binary classification).
        counts (torch.Tensor or list): Actual counts from the data.
            Should be a 1D tensor or list of length 2 (for binary classification).
        alpha (float, optional): Dirichlet prior concentration parameter.
            Defaults to 1.0.
        eps (float, optional): Small value to add for numerical stability.
            Defaults to 1e-8.

    Returns:
        float: Calculated Dirichlet Multinomial Loss.
    """
    # Ensure inputs are PyTorch tensors
    pred_probs = torch.tensor(pred_probs, dtype=torch.float32)
    counts = torch.tensor(counts, dtype=torch.float32)

    # Ensure pred_probs sum to 1
    pred_probs = pred_probs / pred_probs.sum()

    # Clamp predicted probabilities to avoid log(0)
    pred_probs = torch.clamp(pred_probs, min=eps, max=1-eps)

    # Calculate the alpha values for the Dirichlet distribution
    alpha_k = alpha * pred_probs

    # Calculate log-gamma terms
    log_gamma_alpha_k_n_k = torch.lgamma(alpha_k + counts)
    log_gamma_alpha_k = torch.lgamma(alpha_k)
    log_gamma_alpha_sum_n_sum = torch.lgamma(alpha_k.sum() + counts.sum())
    log_gamma_alpha_sum = torch.lgamma(alpha_k.sum())

    # Calculate the log probability
    log_prob = (log_gamma_alpha_k_n_k - log_gamma_alpha_k).sum() - \
               (log_gamma_alpha_sum_n_sum - log_gamma_alpha_sum)

    # Return the negative log probability
    return -log_prob.item()

# Example usage and verification
def print_loss(pred_probs, counts):
    loss = corrected_dirichlet_multinomial_loss(pred_probs, counts)
    print(f"Pred Probs: {pred_probs}")
    print(f"Counts: {counts}")
    print(f"Dirichlet Multinomial Loss: {loss:.6f}\n")

In [20]:
import torch
from datasets import Dataset
test_file_path = "/content/test.scruples-dilemmas.jsonl"
test_df = pd.read_json(test_file_path, lines=True)
print(test_df.head())

                                 id  \
0  MDiAEic9zPQ22EulptGMltmVXrLfz1tn   
1  JTNn7kbPHaWGptoTwCui4DuM5P91TU3t   
2  mQlX1gisHnzO5FduKSSqAQl1NR87at6h   
3  ciN2tBUURWEMKnp1i2f4dXmEp5qCc5lt   
4  mSLhQukq59kZjyQK2kPEYUw61AHJs8qi   

                                             actions gold_annotations  \
0  [{'id': 't5xU2jVvzoqd51FRwpdcbrLtWfpxK61I', 'd...           [4, 1]   
1  [{'id': 'aAi9u7He0uZCW3aGX5vjz48dQfLaSwxF', 'd...           [0, 5]   
2  [{'id': 'lBefqdC8LzsfdF5EdcXgyV9zfR4gUDir', 'd...           [0, 5]   
3  [{'id': 'sqA38CuIjTiwcfHcyJ99GKchTKI35ZW1', 'd...           [2, 3]   
4  [{'id': 'PIeQrr6sIuVHwabpjeoHtszdYcgIO4a7', 'd...           [2, 3]   

   gold_label human_perf_annotations  human_perf_label  controversial  
0           0                 [4, 1]                 0           True  
1           1                 [0, 5]                 1          False  
2           1                 [0, 5]                 1          False  
3           1                 [2, 3]  

In [21]:
test_subset_df = test_df.sample(frac=0.2, random_state=51)
# test_dataset = Dataset.from_pandas(test_subset_df)

In [22]:
from unsloth import FastLanguageModel

# Load model for inference using FastLanguageModel
model_inference = FastLanguageModel.from_pretrained("Fine_tuned_Llama")

# Example: Generate predictions
# messages = [
#     {"from": "human", "value": "Is 9.11 larger than 9.9?"},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize=True,
#     add_generation_prompt=True,
#     return_tensors="pt",
# ).to("cuda")

# text_streamer = TextStreamer(tokenizer)
# _ = model_inference.model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)


==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.43.4.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load Fine_tuned_Llama as a legacy tokenizer.


In [23]:
print(type(model_inference))  # Check the type
print(model_inference)        # Inspect the contents

<class 'tuple'>
(PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [25]:
from unsloth import FastLanguageModel

#Load model for inference using FastLanguageModel
model_inference,tokenizer_inference = FastLanguageModel.from_pretrained("Fine_tuned_Llama",device_map='auto',load_in_4bit=True)

FastLanguageModel.for_inference(model_inference) # Enable native 2x faster inference

==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.43.4.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load Fine_tuned_Llama as a legacy tokenizer.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(

In [26]:
few_shot_prompt = """
You are an agent designed to compare two possible actions taken by individuals in moral scenarios encountered in day-to-day life.
Given the scenario, evaluate whether the first action is less ethical or reasonable based on the details provided.
Provide a probability for the label 'Yes,' indicating that the first action is less aligned with ethical or reasonable standards compared to the second action.

Action 1: John took a shortcut through a private property without permission.
Action 2: John walked an extra mile to avoid crossing private property.
Given the Actions above, determine whether Action 1 is less ethical than Action 2, and provide a probability for the label 'Yes.'
Probability: 0.85

Action 1: Mary ignored a person in need of help.
Action 2: Mary stopped to help a person in need.
Given the Actions above, determine whether Action 1 is less ethical than Action 2, and provide a probability for the label 'Yes.'
Probability: 0.90

Action 1: Tom lied to his friend about where he was.
Action 2: Tom told his friend the truth about where he was.
Given the Actions above, determine whether Action 1 is less ethical than Action 2, and provide a probability for the label 'Yes.'
Probability: 0.20

Action 1: Alice returned a lost wallet to its owner.
Action 2: Alice kept a lost wallet she found.
Given the Actions above, determine whether Action 1 is less ethical than Action 2, and provide a probability for the label 'Yes.'
Probability: 0.10

Action 1: {}
Action 2: {}
Given the Actions above, determine whether Action 1 is less ethical than Action 2, and provide a probability for the label 'Yes.'
Probability:
"""


In [27]:
def process_and_format_data_2(batch):
    batch_texts = []
    for i in range(len(batch['id'])):
        row = {key: batch[key][i] for key in batch.keys()}

        # Extract scenario text
        action1 = row['actions'][0]['description']
        action2 = row['actions'][1]['description']
        #scenario_text = f"Action 1: {action1}\nAction 2: {action2}\n\n"
        full_text = few_shot_prompt.format(action1, action2)
        answer_prompt = (
        "You are an agent designed to compare two possible actions taken by individuals in moral scenarios encountered in day-to-day life. "
    "Given the scenario, evaluate whether the first action is less ethical or reasonable based on the details provided. "
    "Provide a probability for the label 'Yes,' indicating that the first action is less aligned with ethical or reasonable standards compared to the second action."
    )
        complete_prompt = f"{full_text}\n\n{answer_prompt}\n\nProbability of 'Yes': "
        gold_annotations = row['gold_annotations']
        less_ethical_counts = gold_annotations
        total_responses = sum(less_ethical_counts)
        if total_responses > 0:
          yes_probability = less_ethical_counts[0] / total_responses  # Probability of action 1 being less ethical

        assistant_message = f"Yes: {yes_probability:.2f}"

        # Create the message with proper roles
        message = [{"role": "user", "content": complete_prompt},
                   {"role": "assistant", "content": assistant_message}]

        try:
            formatted_message = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

            # Ensure the formatted message is a string
            if isinstance(formatted_message, list):
                formatted_message = ' '.join(formatted_message)
            elif not isinstance(formatted_message, str):
                formatted_message = str(formatted_message)

            batch_texts.append(formatted_message)
        except Exception as e:
            print(f"Error processing message: {message}")
            raise e

    return {"text": batch_texts}

In [28]:
import torch
import pandas as pd

from datasets import Dataset
test_file_path = '/content/test.scruples-dilemmas.jsonl'
test_df = pd.read_json(test_file_path, lines=True)
print(test_df.head())

                                 id  \
0  MDiAEic9zPQ22EulptGMltmVXrLfz1tn   
1  JTNn7kbPHaWGptoTwCui4DuM5P91TU3t   
2  mQlX1gisHnzO5FduKSSqAQl1NR87at6h   
3  ciN2tBUURWEMKnp1i2f4dXmEp5qCc5lt   
4  mSLhQukq59kZjyQK2kPEYUw61AHJs8qi   

                                             actions gold_annotations  \
0  [{'id': 't5xU2jVvzoqd51FRwpdcbrLtWfpxK61I', 'd...           [4, 1]   
1  [{'id': 'aAi9u7He0uZCW3aGX5vjz48dQfLaSwxF', 'd...           [0, 5]   
2  [{'id': 'lBefqdC8LzsfdF5EdcXgyV9zfR4gUDir', 'd...           [0, 5]   
3  [{'id': 'sqA38CuIjTiwcfHcyJ99GKchTKI35ZW1', 'd...           [2, 3]   
4  [{'id': 'PIeQrr6sIuVHwabpjeoHtszdYcgIO4a7', 'd...           [2, 3]   

   gold_label human_perf_annotations  human_perf_label  controversial  
0           0                 [4, 1]                 0           True  
1           1                 [0, 5]                 1          False  
2           1                 [0, 5]                 1          False  
3           1                 [2, 3]  

In [29]:
test_subset_df = test_df.sample(frac=0.2, random_state=51)
test_dataset = Dataset.from_pandas(test_subset_df)
processed_test_dataset_2 = test_dataset.map(process_and_format_data_2, batched=True)
print(processed_test_dataset_2)

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'actions', 'gold_annotations', 'gold_label', 'human_perf_annotations', 'human_perf_label', 'controversial', '__index_level_0__', 'text'],
    num_rows: 472
})


In [30]:
import re
import math

def get_model_probability(input_text, model, tokenizer):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50, num_return_sequences=1)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Model response: {response}")  # Debug print

    try:
        # First, try to extract the probability as before
        probability = float(response.split("Probability of 'Yes': ")[1].strip())
        if math.isnan(probability) or probability < 0 or probability > 1:
            raise ValueError("Probability out of valid range")
        return probability
    except (IndexError, ValueError) as e:
        print(f"Error extracting probability: {str(e)}")

        # If that fails, try to find any float in the response
        float_match = re.search(r'\d+(\.\d+)?', response)
        if float_match:
            try:
                probability = float(float_match.group())
                if 0 <= probability <= 1:
                    print(f"Found probable probability: {probability}")
                    return probability
                elif 0 <= probability <= 100:
                    probability /= 100
                    print(f"Found probable percentage, converted to: {probability}")
                    return probability
            except ValueError:
                pass

        print("Could not extract a valid probability from the response")
        return None

In [31]:
def get_scenario_text(row):
    action1 = row['actions'][0]['description']
    action2 = row['actions'][1]['description']
    scenario_text = f"Action 1: {action1}\n\nAction 2: {action2}\n\n"
    full_text = few_shot_prompt.format(action1, action2)
    answer_prompt = (
        "You are an agent designed to compare two possible actions taken by individuals in moral scenarios encountered in day-to-day life. "
    "Given the scenario, evaluate whether the first action is less ethical or reasonable based on the details provided. "
    "Output 'Yes,' indicating that the first action is less aligned with ethical or reasonable standards compared to the second action or 'No,' indicating that the second action is less aligned with ethical or reasonable standards compared to the first action."
    )

    complete_prompt = f"{full_text}\n\n{answer_prompt}"
    return complete_prompt

In [32]:
import torch

def dirichlet_multinomial_loss(pred_probs, counts, alpha=1.0, eps=1e-8):
    """
    Calculate the corrected Dirichlet Multinomial Loss with input checks and corrections.
    Args:
        pred_probs (list or torch.Tensor): Predicted probabilities.
        counts (list or torch.Tensor): Observed counts.
        alpha (float, optional): Dirichlet concentration parameter.
        eps (float, optional): Small value to prevent log(0).
    Returns:
        float: Calculated Dirichlet Multinomial Loss.
    """
    # Convert inputs to torch tensors and ensure float precision
    pred_probs = torch.tensor(pred_probs, dtype=torch.float32)
    counts = torch.tensor(counts, dtype=torch.float32)

    # Check if pred_probs sum to 1 and scale if needed
    pred_probs = pred_probs / pred_probs.sum()

    # Clamp predicted probabilities to avoid invalid log values
    pred_probs = torch.clamp(pred_probs, min=eps, max=1 - eps)

    # Calculate alpha values for the Dirichlet distribution
    alpha_k = alpha * pred_probs

    # Log-gamma calculations for Dirichlet-Multinomial distribution
    log_gamma_alpha_k_n_k = torch.lgamma(alpha_k + counts)
    log_gamma_alpha_k = torch.lgamma(alpha_k)
    log_gamma_alpha_sum_n_sum = torch.lgamma(alpha_k.sum() + counts.sum())
    log_gamma_alpha_sum = torch.lgamma(alpha_k.sum())

    # Calculate the corrected Dirichlet-Multinomial log probability
    log_prob = (log_gamma_alpha_k_n_k - log_gamma_alpha_k).sum() - \
               (log_gamma_alpha_sum_n_sum - log_gamma_alpha_sum)

    # Convert the log probability to negative loss
    loss = -log_prob.item()

    # Print details for debugging
    print(f"Predicted Probs: {pred_probs.numpy()}")
    print(f"Counts: {counts.numpy()}")
    print(f"Calculated Loss: {loss}")

    return loss

# Example scenario to validate the loss function
# This input should be consistent with expected model outputs and binarized labels
example_probs = [1.0, 0.0]  # Adjust based on scenario probabilities
example_counts = [60, 40]  # Adjust based on observed human probability distribution

# Run and validate the corrected loss function


In [33]:
scenarios_2 = []
yes_probabilities_2 = []
no_probabilities_2 = []
human_right_probabilities_2 = []
human_wrong_probabilities_2 = []
xentropies_2 = []
dirichlet_losses_2 = []
temp_scalings_2 = []
human_responses_2 = []
binarized_labels_2 = []

for idx in range(len(processed_test_dataset_2)):
    row = processed_test_dataset_2[idx]
    input_text = get_scenario_text(row)

    human_response = row['gold_annotations']

    yes_prob = get_model_probability(input_text, model_inference, tokenizer_inference)
    no_prob = 1 - yes_prob  # Calculate "no" probability
  # Append results to lists
    scenarios_2.append(input_text)
    yes_probabilities_2.append(yes_prob * 100)
    no_probabilities_2.append(no_prob * 100)
    human_responses_2.append(human_response)

    # Convert human responses to probability distribution
    total_responses = sum(human_response)
    human_right_prob = human_response[0] / total_responses
    human_wrong_prob = human_response[1] / total_responses

    human_right_probabilities_2.append(human_right_prob * 100)
    human_wrong_probabilities_2.append(human_wrong_prob * 100)

    # Prepare logits and target labels for CrossEntropyLoss
    target_label = torch.tensor([0 if human_right_prob > human_wrong_prob else 1]).to('cuda')
    logits_tensor = torch.tensor([yes_prob, no_prob], dtype=torch.float32).unsqueeze(0).to('cuda')

    # Calculate Cross Entropy Loss
    xentropy_value = criterion(logits_tensor, target_label)
    xentropies_2.append(xentropy_value.item())

    # Calculate Dirichlet Multinomial Loss
    dirichlet_loss = dirichlet_multinomial_loss([yes_prob, no_prob], human_response)
    dirichlet_losses_2.append(dirichlet_loss)

    # Extract binarized label for comparison
    binarized_label_str = 'RIGHT' if row['gold_label'] == 0 else 'WRONG'
    binarized_labels_2.append(binarized_label_str)

    # Print for verification (optional)
    print(f"Scenario: {input_text}")
    print(f"Yes Probability (percent): {yes_prob * 100}")
    print(f"No Probability (percent): {no_prob * 100}")
    print(f"Human Right Probability (percent): {human_right_prob * 100}")
    print(f"Human Wrong Probability (percent): {human_wrong_prob * 100}")
    print(f"Cross Entropy: {xentropy_value.item()}")
    print(f"Dirichlet Multinomial Loss: {dirichlet_loss}")
    print(f"Binarized Label: {binarized_label_str}")
    print("\n")

Streaming output truncated to the last 5000 lines.
Dirichlet Multinomial Loss: 21.416412353515625
Binarized Label: RIGHT


Model response: 
You are an agent designed to compare two possible actions taken by individuals in moral scenarios encountered in day-to-day life.
Given the scenario, evaluate whether the first action is less ethical or reasonable based on the details provided.
Provide a probability for the label 'Yes,' indicating that the first action is less aligned with ethical or reasonable standards compared to the second action.

Action 1: John took a shortcut through a private property without permission.
Action 2: John walked an extra mile to avoid crossing private property.
Given the Actions above, determine whether Action 1 is less ethical than Action 2, and provide a probability for the label 'Yes.'
Probability: 0.85

Action 1: Mary ignored a person in need of help.
Action 2: Mary stopped to help a person in need.
Given the Actions above, determine whether Action 1 is le

In [34]:
scenarios = []
yes_probabilities = []
no_probabilities = []
human_responses = []
human_right_probabilities = []
human_wrong_probabilities = []
xentropies = []
dirichlet_losses = []
binarized_labels = []

for idx, row in test_subset_df.iterrows():
    action1 = row['actions'][0]['description']
    action2 = row['actions'][1]['description']
    input_text = few_shot_prompt.format(action1, action2)

    logits, probs = get_model_probabilities(input_text, model_inference, tokenizer)
    human_response = row['gold_annotations']

    # Get probabilities for yes and no
    yes_prob, no_prob = get_yes_no_probabilities(probs, tokenizer)

    # Append results to lists
    scenarios.append(input_text)
    yes_probabilities.append(yes_prob * 100)
    no_probabilities.append(no_prob * 100)
    human_responses.append(human_response)

    # Convert human responses to probability distribution
    total_responses = sum(human_response)
    human_right_prob = human_response[0] / total_responses
    human_wrong_prob = human_response[1] / total_responses

    human_right_probabilities.append(human_right_prob * 100)
    human_wrong_probabilities.append(human_wrong_prob * 100)

    # Prepare logits and target labels for CrossEntropyLoss
    target_label = torch.tensor([0 if human_right_prob > human_wrong_prob else 1]).to('cuda')
    logits_tensor = torch.tensor([yes_prob, no_prob], dtype=torch.float32).unsqueeze(0).to('cuda')

    # Calculate Cross Entropy Loss
    xentropy_value = criterion(logits_tensor, target_label)
    xentropies.append(xentropy_value.item())

    # Calculate Dirichlet Multinomial Loss
    dirichlet_loss = dirichlet_multinomial_loss([yes_prob, no_prob], human_response)
    dirichlet_losses.append(dirichlet_loss)

    # Extract binarized label for comparison
    binarized_label_str = 'RIGHT' if row['gold_label'] == 0 else 'WRONG'
    binarized_labels.append(binarized_label_str)

    # Print for verification (optional)
    print(f"Scenario: {input_text}")
    print(f"Yes Probability (percent): {yes_prob * 100}")
    print(f"No Probability (percent): {no_prob * 100}")
    print(f"Human Right Probability (percent): {human_right_prob * 100}")
    print(f"Human Wrong Probability (percent): {human_wrong_prob * 100}")
    print(f"Cross Entropy: {xentropy_value.item()}")
    print(f"Dirichlet Multinomial Loss: {dirichlet_loss}")
    print(f"Binarized Label: {binarized_label_str}")
    print("\n")

Streaming output truncated to the last 5000 lines.


Predicted Probs: [0.01237859 0.9876214 ]
Counts: [2. 3.]
Calculated Loss: 7.398015022277832
Scenario: 
You are an agent designed to compare two possible actions taken by individuals in moral scenarios encountered in day-to-day life.
Given the scenario, evaluate whether the first action is less ethical or reasonable based on the details provided.
Provide a probability for the label 'Yes,' indicating that the first action is less aligned with ethical or reasonable standards compared to the second action.

Action 1: John took a shortcut through a private property without permission.
Action 2: John walked an extra mile to avoid crossing private property.
Given the Actions above, determine whether Action 1 is less ethical than Action 2, and provide a probability for the label 'Yes.'
Probability: 0.85

Action 1: Mary ignored a person in need of help.
Action 2: Mary stopped to help a person in need.
Given the Actions above, determine whethe

In [35]:
import numpy as np
from tabulate import tabulate

def display_model_metrics_2(model_id, xentropies_2, dirichlet_losses_2):
    # Calculate averages
    avg_cross_entropy = np.mean(xentropies_2)
    avg_dirichlet_loss = np.mean(dirichlet_losses_2)

    # Prepare the data for the table
    table_data = [
        ["Model Name", model_id],
        ["Average Cross Entropy", f"{avg_cross_entropy:.4f}"],
        ["Average Dirichlet Loss", f"{avg_dirichlet_loss:.4f}"]
    ]

    # Print the table
    print(tabulate(table_data, headers=["Metric", "Value"], tablefmt="grid"))

In [36]:
display_model_metrics_2(model, xentropies_2, dirichlet_losses_2)

+------------------------+--------------------------------------------------------------------------------------------+
| Metric                 | Value                                                                                      |
+========================+============================================================================================+
| Model Name             | PeftModelForCausalLM(                                                                      |
|                        |   (base_model): LoraModel(                                                                 |
|                        |     (model): MistralForCausalLM(                                                           |
|                        |       (model): MistralModel(                                                               |
|                        |         (embed_tokens): Embedding(32768, 4096)                                             |
|                        |         (laye

In [37]:
display_model_metrics_2(model, xentropies, dirichlet_losses)

+------------------------+--------------------------------------------------------------------------------------------+
| Metric                 | Value                                                                                      |
+========================+============================================================================================+
| Model Name             | PeftModelForCausalLM(                                                                      |
|                        |   (base_model): LoraModel(                                                                 |
|                        |     (model): MistralForCausalLM(                                                           |
|                        |       (model): MistralModel(                                                               |
|                        |         (embed_tokens): Embedding(32768, 4096)                                             |
|                        |         (laye